# Interface to FMU Proxy
> This module contains classes and methods that provides user-friendly interfaces to manage
an FMU proxy server

In [2]:
#| default_exp fmu_proxy
%load_ext autoreload
%autoreload 2

In [3]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
import logging
from dataclasses import dataclass
from subprocess import Popen
from typing import Dict, Union


In [ ]:
#| export


@dataclass
class NetworkEndpoint:
    """Namedtuple class for a network endpoint"""
    address: str
    port: int

    def to_dict(self) -> Dict[str, Union[str, int]]:
        """Return dictionary form of the data"""
        return {'address': self.address, 'port': self.port}

    def to_json(self) -> str:
        """Return json form of the data"""
        return json.dumps(self.to_dict())

    @property
    def network_string(self) -> str:
        """Returns a network string"""
        return f"{self.address}:{self.port}"


class DistributedSimulationProxyServer:
    """Class for handling distributed simulation proxy server"""

    def __init__(
            self,
            file_path_fmu: str,
            guid: str = None,
            endpoint: NetworkEndpoint = NetworkEndpoint(address="localhost", port=9090),
    ):
        """Constructor for the class"""
        self.endpoint = endpoint
        self.guid = guid
        self.file_path_fmu = file_path_fmu

    @property
    def _query_string(self):
        """Returns the query string for the endpoint"""
        return f"file={self.file_path_fmu}" if self.guid is None else f"guid={self.guid}"

    @property
    def endpoint_str(self):
        """Returns the end point for a system structure file"""
        return f"fmu-proxy://{self.endpoint.network_string}?{self._query_string}"

    def run_local_proxy_server_async(
            self,
            path_to_server_executable,
            logger: logging.Logger = None,
    ) -> Union[Popen, bool]:
        """Run a local proxy server if it is set to be local. Only applicable for java based proxy server"""
        if self.endpoint.address != "localhost":
            if logger is not None:
                logger.error("This instance is for remote proxy server. Skip the command")
            return False

        args = [
            'java', '-jar',
            path_to_server_executable, '-thrift/tcp', str(self.endpoint.port),
            self.file_path_fmu
        ]
        if logger is not None:
            logger.info(f"Starting a proxy-server at port {self.endpoint.port}.")
            logger.info(" ".join(args))
        return Popen(args=args)
